# **Reddit Scraper**

In [1]:
!pip install praw

In [40]:
# Import packages
import praw
import psaw
import pandas as pd
import numpy as np
import datetime as dt
from textblob import TextBlob

In [8]:
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 50)

In [65]:
# Access Reddit API 
reddit = praw.Reddit(client_id= "ibh9FWjQqVYZYw",                      # your client id
                     client_secret= "VeGeOHgpCN9SVSDsClpaR_IRWsCfiA",  # your client secret
                     user_agent= "reddit_scraper",                     # user agent (app) name
                     username = "asharm2949",                          # your reddit username asharm2949
                     password = "aman-sharma2949",                     # your reddit password
                     check_for_async=False)               

# Subreddit name and queries list to srape
scrape_dict = {
                'CryptoCurrency': ['GameFi', 'P2E'],
                'CryptoMarkets': ['GameFi', 'P2E'],
                'CryptoCurrencies': ['GameFi', 'P2E'],
                'AxieInfinity': ['GameFi', 'P2E'],
                'DefiKingdoms': ['GameFi', 'P2E'],
            
}

scrape_list = ['AxieInfinity', 'DefiKingdoms', 'NFT']

# Helper function 
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [84]:
def scrape_reddit(scrape_dict):
  
  # Post data dictionary
  post_dict = {
      "timestamp_scrape" : [],  # timestamp of the scrape
      "date_scrape" : [],       # date of the scrape    
      "timestamp_post" : [],    # timestamp of the post
      "subreddit": [],          # subreddit of the post
      "query": [],              # requested query 
      "sort_type": [],          # requested reddit sort type  
      "id" : [],                # unique id of the post
      "title" : [],             # title of the post
      "url" : [],               # url of the post
      "body" : [],              # the descriptionof post
      "score" : [],             # number of upvotes of the post
      "upvote_ratio": [],       # ratio of upvotes to total votes
      "num_comms": [],          # the number of comments on the post
  }

  # Type of posts to scrape
  sort_list = ['new']

  # Scrape data 
  for key, value in scrape_dict.items():
    subreddit = reddit.subreddit(key)   

    for query in value:
        
      for sort in sort_list:

        for submission in subreddit.search(query, sort = sort, limit = 10000):
          post_dict['timestamp_scrape'].append(pd.to_datetime("now"))
          post_dict['date_scrape'].append(dt.date.today())
          post_dict["timestamp_post"].append(submission.created_utc)
          post_dict["subreddit"].append(submission.subreddit)
          post_dict["query"].append(query)
          post_dict["sort_type"].append(sort)
          post_dict["id"].append(submission.id)
          post_dict["title"].append(submission.title)
          post_dict["url"].append(submission.url)
          post_dict["body"].append(submission.selftext)
          post_dict["score"].append(submission.score)
          post_dict["upvote_ratio"].append(submission.upvote_ratio)           
          post_dict["num_comms"].append(submission.num_comments)


  reddit_data = pd.DataFrame(post_dict)
  reddit_data['timestamp_post'] = reddit_data["timestamp_post"].apply(get_date)
  reddit_data.drop_duplicates(subset = ['date_scrape', 'timestamp_post', 'subreddit', 'sort_type', 'id', 'title', 'url', 'body', 'score', 'upvote_ratio', 'num_comms'])

  return reddit_data


In [85]:
def scrape_reddit_mass(scrape_list):
  
  # Post data dictionary
  post_dict = {
      "timestamp_scrape" : [],  # timestamp of the scrape
      "date_scrape" : [],       # date of the scrape    
      "timestamp_post" : [],    # timestamp of the post
      "subreddit": [],          # subreddit of the post
      "id" : [],                # unique id of the post
      "title" : [],             # title of the post
      "url" : [],               # url of the post
      "body" : [],              # the descriptionof post
      "score" : [],             # number of upvotes of the post
      "upvote_ratio": [],       # ratio of upvotes to total votes
      "num_comms": [],          # the number of comments on the post
  }

  for subreddit in scrape_list:
    posts = reddit.subreddit(str(subreddit)).hot(limit=5000)
    
    for post in posts:
      post_dict['timestamp_scrape'].append(pd.to_datetime("now"))
      post_dict['date_scrape'].append(dt.date.today())
      post_dict["timestamp_post"].append(post.created_utc)
      post_dict["subreddit"].append(post.subreddit)
      post_dict["id"].append(post.id)
      post_dict["title"].append(post.title)
      post_dict["url"].append(post.url)
      post_dict["body"].append(post.selftext)
      post_dict["score"].append(post.score)
      post_dict["upvote_ratio"].append(post.upvote_ratio)           
      post_dict["num_comms"].append(post.num_comments)

  reddit_data_mass = pd.DataFrame(post_dict)
  reddit_data_mass['timestamp_post'] = reddit_data_mass["timestamp_post"].apply(get_date)

  return reddit_data_mass

In [86]:
test = scrape_reddit(scrape_dict)
print(test.shape)
# sample_scrape = scrape_reddit(scrape_dict)
# print(sample_scrape.shape)
# sample_scrape.head(5)

(897, 13)


In [87]:
test.tail(10)

,timestamp_scrape,date_scrape,timestamp_post,subreddit,query,sort_type,id,title,url,body,score,upvote_ratio,num_comms
887,2022-12-06 13:08:37.455546,2022-12-06,2022-01-18 15:54:55,DefiKingdoms,P2E,new,s77ma5,Some help for a fellow crypto enthusiast?,https://www.reddit.com/r/DefiKingdoms/comments/s77ma5/some_help_for_a_fellow_crypto_enthusiast/,"Hello everyone,\n\nI'm a Brazilian young man that has been growing much interest with the crypto world (blockchains, p2e, NFT...). I've known about cryptocurrencies from a long time, but hasn't been much interested before cause I always found it to be much of a speculative area to do any major investment. However, with the recently mixture of games with crypto I believe we all have a more tangible approach on that speculation. We're now able to judge many aspects of a game and it's respectiv...",5,0.78,14
888,2022-12-06 13:08:37.456056,2022-12-06,2022-01-04 16:48:23,DefiKingdoms,P2E,new,rw5pah,Beginner's guide to buying the first hero,https://www.reddit.com/r/DefiKingdoms/comments/rw5pah/beginners_guide_to_buying_the_first_hero/,"Ofc, none of this is financial advice.\n\nDisclaimer: this guide assumes that you know the basics of working with a DEX (yield farming, providing liquidity, impermanent loss).\n\n&#x200B;\n\n**Who am I?**\n\nI am a self proclaimed disciple of Taiki Maeda and first learned about DFK through his discord group. I was lucky enough to ape in during Gen 0 minting and have been participating in the hero/gaming side of DFK since then. Now I run a small private community/DAO dedicated to DFK. \n\n**W...",202,1.00,75
889,2022-12-06 13:08:37.456547,2022-12-06,2021-12-28 16:23:26,DefiKingdoms,P2E,new,rqq32k,Barrier to entry and Avax,https://www.reddit.com/r/DefiKingdoms/comments/rqq32k/barrier_to_entry_and_avax/,"I’ve been playing DFK for about 6 weeks now and I totally love it! I think the p2e is getting better day by day. I have 2 important questions though.\n\n1. The barrier of entry gets higher each day. 6 weeks ago the cheapest hero was about 200$, now it’s 500$. A lot of new players won’t enter the game with this amount knowing they will also get the worst hero possible. Possible solution: give a free hero linked to a wallet address which gives very limited rewards but let’s people experience t...",9,0.76,11
890,2022-12-06 13:08:37.456989,2022-12-06,2021-12-21 12:40:15,DefiKingdoms,P2E,new,rljuep,Proposal: a free non-NFT/non-tradable hero for all,https://www.reddit.com/r/DefiKingdoms/comments/rljuep/proposal_a_free_nonnftnontradable_hero_for_all/,"Goal of this post: Create a discussion about a solution to the barrier entry cost.\n\n\nGoal of this proposal: Lower the barrier cost of playing DFK to allow essentially anyone to play the game.\n\nProposal: Everyone, current and future players will receive a non-NFT, non-tradable hero to start off with. This starting hero will be limited in many ways, as to deter players from not taking advantage of this free hero, but also to bring in new players whether they want to try out the game or is...",35,0.76,43
891,2022-12-06 13:08:37.457264,2022-12-06,2021-12-12 18:39:01,DefiKingdoms,P2E,new,rf1gup,Why shouldn't you provide liquidity to DFK?,https://www.reddit.com/r/DefiKingdoms/comments/rf1gup/why_shouldnt_you_provide_liquidity_to_dfk/,"As it stands, I currently do not see any drawbacks to putting a good portion, if not all, of my BTC and ETH into a DFK LP. Particularly, the BTC/ETH pool. Even though its APR is the lowest, the risk is lower than the others. If one currency rises, I receive the other to keep the 1:1 ratio. I'm happy with both. Whereas, if I provide liquidity to the Jewel/ETH pool and jewel dumps, I lose my ETH.\n\nI'm curious to know what the all-together risk is with this play. Can DFK get hacked? If so, w...",17,0.91,26
892,2022-12-06 13:08:37.457498,2022-12-06,2021-12-01 02:24:09,DefiKingdoms,P2E,new,r69561,👑 DeFi Kingdoms Quests Live to Production 👑,https://www.reddit.com/r/DefiKingdoms/comments/r69561/defi

In [76]:
polarity_arr = []
for i in np.arange(0, len(test)):
    polarity_arr.append(TextBlob(test['title'][i]).sentiment.polarity)

In [77]:
sum(polarity_arr)/len(polarity_arr)

0.05283673635837097

In [78]:
subjectivity_arr = []
for i in np.arange(0, len(test)):
    subjectivity_arr.append(TextBlob(test['title'][i]).sentiment.subjectivity)

In [79]:
sum(subjectivity_arr)/len(subjectivity_arr)

0.2828962592775503